In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


In [3]:
Batch_size = 32
picture_size=150
datagen_train = ImageDataGenerator(
    rescale=1./255,        # Normalize image pixel values, Normalizes pixel values by scaling them between 0 and 1
    rotation_range=20,      # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20%
    height_shift_range=0.2, # Randomly shift images vertically by 20%
    shear_range=0.2,        # Apply random shear transformations
    zoom_range=0.2,         # Randomly zoom in/out by up to 20%
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Fill in missing pixels after transformation
)



train_set = datagen_train.flow_from_directory("train",
                                                 target_size = (picture_size, picture_size),
                                                 color_mode = 'grayscale',
                                                 batch_size = Batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle = True)

datagen_test = ImageDataGenerator(rescale=1./255)



test_set = datagen_test.flow_from_directory("validation",
                                                 target_size = (picture_size, picture_size),
                                                 color_mode = 'grayscale',
                                                 batch_size = Batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle = False)
                                                               

Found 28385 images belonging to 6 classes.
Found 6955 images belonging to 6 classes.


In [4]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(6, activation='softmax'))  

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [5]:
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 74, 74, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 36, 36, 128)       0

In [6]:
# Fit the model
modelv1 = model.fit(
    train_set,
    epochs=30,                     # You can adjust the number of epochs
    validation_data=test_set,       # Validation data for evaluating model performance on unseen data
    steps_per_epoch=train_set.samples // Batch_size,  # Number of steps per epoch
    validation_steps=test_set.samples // Batch_size   # Validation steps for each epoch
)

# Optional: Save the trained model
model.save('emotion_recognition_model.h5')


Epoch 1/30
115/887 [==>...........................] - ETA: 18:17 - loss: 1.7829 - accuracy: 0.2443

KeyboardInterrupt: 